In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

england_matches = pd.read_json('new_data/matches/matches_England.json')
teams = pd.read_json('new_data/teams.json')
england_events = pd.read_json('new_data/events/events_England.json')
players = pd.read_json('new_data/players.json')

In [2]:
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
from IPython.display import IFrame
import json

In [6]:
class team:
    def __init__(self, name:str):
        self.name = name
        self.team_id = self.get_team_id()
        self.matches = self.get_matches()
        self.players = self.get_players()
        self.all_events = self.get_all_events()
    
    def get_team_id(self):
        return int(teams[teams['name'] == self.name]['wyId'])
        
    def get_matches(self, opposition_team:team=None):
        matches = pd.DataFrame(columns=england_matches.columns)
        specific_matches = pd.DataFrame(columns=england_matches.columns)
        
        for index,row in england_matches.iterrows():
            teams_in_match = [int(team) for team in list(row['teamsData'].keys())]
            if opposition_team is None:
                if self.team_id in teams_in_match:
                    matches = matches.append(row)
            else:
                if self.team_id in teams_in_match and opposition_team.team_id in teams_in_match:
                    specific_matches = specific_matches.append(row)
                
        if opposition_team is None:     
            return matches
        else:
            return specific_matches
    
    def get_players(self):
        return players[players['currentTeamId'] == self.team_id]
    
    def get_players_in_match(self, match_id:int):
        target_match_events = self.all_events[self.all_events['matchId'] == match_id]
        target_match_player_ids = list(set(target_match_events['playerId'].tolist()))
        target_match_players = self.players[self.players['wyId'].isin(target_match_player_ids)]
        return target_match_players
    
    def get_all_events(self):
        return england_events[england_events['teamId'] == self.team_id]

In [9]:
from ipywidgets import interact, interactive, interact_manual, widgets, Layout
import warnings 
warnings.filterwarnings('ignore')

def dashboard_graph():
    #global variables whose values will be updated with every successive selection : filtering achieved through modification of these global vars
    team_1, team_2 = None, None
    matches = None
    target_match = None
    target_team = None
    target_player = None
    events = None
    selected_events = None
    teamData = None
    target_match_not = None

    #First selector that user sees is team selector : get its options
    england_teams = pd.DataFrame(columns=teams.columns)
    for index,row in teams.iterrows():
        if row['area']['name'] == 'England':
            england_teams = england_teams.append(row)
    team_selector_options = sorted(england_teams['name'].tolist())

    #create team selectors - first interactive element to be called, invocation is on last line 
    team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
    team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

    def select_teams(team_1_selector, team_2_selector):
        global team_1, team_2, matches

        team_1 = team(team_1_selector)
        team_2 = team(team_2_selector)
        matches = team_1.get_matches(team_2)

        #teams selected, call match selector to select one of 2 matches between them
        match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match:', layout=Layout(margin='0px 0px 0px -3px'))
        interact(select_match, match_selector=match_selector)

    def select_match(match_selector) -> pd.DataFrame:
        global matches, target_match, team_1, team_2, target_match_not

        target_match = matches[matches['label'] == match_selector]
        target_match_not = matches[matches['label'] != match_selector]
#         display(target_match) #for debugging
    
        #match selected, select which team in match to analyse
        target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team:', layout=Layout(margin='0px 0px 0px -8px'))
        interact(select_team, team_selector=target_team_selector)
        
    def select_team(team_selector):
        global team_1, team_2, target_match, target_team, events, target_match_not

        target_team = team_1 if team_selector == team_1.name else team_2

        events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])] 
        events_not = target_team.all_events[target_team.all_events['matchId'] == int(target_match_not['wyId'])]
        print(target_match.iloc[0]['label'])
        print(target_match_not.iloc[0]['label'])

#         print(events.shape)
#         print(events_not.shape)
        
#         print(events.head())
#         teamId = events['teamId'][0:1]
        teamId = events.iloc[0]['teamId']
#         print('Team ID', teamId)
#         print(target_match.iloc[0]['teamsData'])
        lineup = target_match.iloc[0]['teamsData']
#         print(lineup[str(teamId)]['formation']['lineup'])
        first_11 = lineup[str(teamId)]['formation']['lineup']
    
#         first_11 = team_info['1609']['formation']['lineup']
        players_id = []
        players_dict = {}
        players_dict_ulta = {}
        for i in range(len(first_11)):
        #     print(first_11[i]['playerId'])
            players_id.append(first_11[i]['playerId'])
            players_dict[first_11[i]['playerId']] = i
            players_dict_ulta[i] = first_11[i]['playerId']

#         print(players_id)
#         print(players_dict)
#         print(players_dict_ulta)

        players_names = {}
        for i in range(11):
            pid = players_dict_ulta[i]
            players_names[i] = players[players['wyId']==pid].iloc[0]['shortName']

#         print(players_names)
        
        team_pass = events[events['eventName']=='Pass']
        team_pass = team_pass[team_pass['playerId'].isin(players_id)]
        team_pass['Length'] = team_pass.tags.apply(lambda x: len(x))
        team_pass = team_pass[team_pass['Length']==1]
        team_pass['accurate'] = team_pass.apply(lambda row: row['tags'][0]['id'], axis=1)
        team_pass = team_pass[team_pass['accurate']==1801]
        team_pass = team_pass.reset_index(drop=True)
#         print(team_pass.head())
        
        players_matrix = np.zeros((11, 11), dtype=int)
        
        for i in range(1, len(team_pass)):
            #print(arsenal_pass.loc[i, 'eventSec']-arsenal_pass.loc[i-1, 'eventSec'])
            if team_pass.loc[i, 'eventSec']-team_pass.loc[i-1, 'eventSec'] < 5.0:
                #print(arsenal_pass.loc[i, 'eventSec']-arsenal_pass.loc[i-1, 'eventSec'])
                player_from = team_pass.loc[i-1, 'playerId']
                player_to = team_pass.loc[i, 'playerId']
                #print(player_from, player_to)
#                 print(players_dict[player_from], players_dict[player_to])
                if player_from != player_to:
                    players_matrix[players_dict[player_from]][players_dict[player_to]] += 1
                    players_matrix[players_dict[player_to]][players_dict[player_from]] += 1
                    
#         print(players_matrix)
        
        G = nx.from_numpy_matrix(players_matrix)
        
        G = nx.relabel_nodes(G, players_names)
        
#         print(G.nodes())
        
        nx.draw(G, with_labels=True)   #Does not show anything
        
#         print(list(G.edges(data=True)))

        cent = nx.algorithms.centrality.betweenness_centrality(G, weight='weight')
#         print(cent)
        
        nx.set_node_attributes(G, cent, 'betweenness')
        
        clus = nx.clustering(G, weight='weight')
#         print(clus)
        
        nx.set_node_attributes(G, clus, 'Cluster')
        
#         print(G.nodes(data=True))
        
        team_json = json_graph.node_link_data(G)
#         print(team_json)
        
        with open("sample_weight_clus.json", "w") as outfile: 
            json.dump(team_json, outfile)
            
        display(IFrame(src='./d3_tip.html', width=900, height=600))
#         display(IFrame(src='./d3_tip.html', width=900, height=600))



    #invoke team selector : get the ball rolling
    interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

In [10]:
dashboard_graph()

A Jupyter Widget

## Trying to compare away vs home for a team

In [13]:
from ipywidgets import interact, interactive, interact_manual, widgets, Layout
import warnings 
warnings.filterwarnings('ignore')

def dashboard_graph_both():
    #global variables whose values will be updated with every successive selection : filtering achieved through modification of these global vars
    team_1, team_2 = None, None
    matches = None
    target_match = None
    target_team = None
    target_player = None
    events = None
    selected_events = None
    teamData = None
    target_match_not = None

    #First selector that user sees is team selector : get its options
    england_teams = pd.DataFrame(columns=teams.columns)
    for index,row in teams.iterrows():
        if row['area']['name'] == 'England':
            england_teams = england_teams.append(row)
    team_selector_options = sorted(england_teams['name'].tolist())

    #create team selectors - first interactive element to be called, invocation is on last line 
    team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
    team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

    def select_teams(team_1_selector, team_2_selector):
        global team_1, team_2, matches

        team_1 = team(team_1_selector)
        team_2 = team(team_2_selector)
        matches = team_1.get_matches(team_2)

        #teams selected, call match selector to select one of 2 matches between them
        match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match:', layout=Layout(margin='0px 0px 0px -3px'))
        interact(select_match, match_selector=match_selector)

    def select_match(match_selector) -> pd.DataFrame:
        global matches, target_match, team_1, team_2, target_match_not

        target_match = matches[matches['label'] == match_selector]
        target_match_not = matches[matches['label'] != match_selector]
#         display(target_match) #for debugging
    
        #match selected, select which team in match to analyse
        target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team:', layout=Layout(margin='0px 0px 0px -8px'))
        interact(select_team, team_selector=target_team_selector)
        
    def select_team(team_selector):
        global team_1, team_2, target_match, target_team, events, target_match_not

        target_team = team_1 if team_selector == team_1.name else team_2
        
#         print('Before Events')
#         print('Target', target_match.iloc[0]['label'])
#         print('Target_not', target_match_not.iloc[0]['label'])

        events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])] 
        events_not = target_team.all_events[target_team.all_events['matchId'] == int(target_match_not['wyId'])]
        events_dummy = events
        target_match_dummy = target_match
#         print('After Events')
#         print('Target', target_match.iloc[0]['label'])
#         print('Target_not', target_match_not.iloc[0]['label'])
        
        for i in range(2):
            
            if i==1:
                events_new = events_not 
                target_match_new = target_match_not
                print(target_match_new.iloc[0]['label'])
            else:
                events_new = events_dummy 
                target_match_new = target_match_dummy
                print(target_match_new.iloc[0]['label'])
            
#             print(i, target_match.iloc[0]['label'])
            
            teamId = events_new.iloc[0]['teamId']
#             print(teamId)

            lineup = target_match_new.iloc[0]['teamsData']

            first_11 = lineup[str(teamId)]['formation']['lineup']

            players_id = []
            players_dict = {}
            players_dict_ulta = {}
            for i in range(len(first_11)):

                players_id.append(first_11[i]['playerId'])
                players_dict[first_11[i]['playerId']] = i
                players_dict_ulta[i] = first_11[i]['playerId']

            players_names = {}
            for i in range(11):
                pid = players_dict_ulta[i]
                players_names[i] = players[players['wyId']==pid].iloc[0]['shortName']

            team_pass = events_new[events_new['eventName']=='Pass']
            team_pass = team_pass[team_pass['playerId'].isin(players_id)]
            team_pass['Length'] = team_pass.tags.apply(lambda x: len(x))
            team_pass = team_pass[team_pass['Length']==1]
            team_pass['accurate'] = team_pass.apply(lambda row: row['tags'][0]['id'], axis=1)
            team_pass = team_pass[team_pass['accurate']==1801]
            team_pass = team_pass.reset_index(drop=True)

            players_matrix = np.zeros((11, 11), dtype=int)

            for i in range(1, len(team_pass)):

                if team_pass.loc[i, 'eventSec']-team_pass.loc[i-1, 'eventSec'] < 5.0:

                    player_from = team_pass.loc[i-1, 'playerId']
                    player_to = team_pass.loc[i, 'playerId']

                    if player_from != player_to:
                        players_matrix[players_dict[player_from]][players_dict[player_to]] += 1
                        players_matrix[players_dict[player_to]][players_dict[player_from]] += 1


            G = nx.from_numpy_matrix(players_matrix)

            G = nx.relabel_nodes(G, players_names)

            nx.draw(G, with_labels=True)   #Does not show anything

            cent = nx.algorithms.centrality.betweenness_centrality(G, weight='weight')
            
            avg_cent = 0
            for i in cent:
                avg_cent += cent[i]
            print('Average Betwenness = ', avg_cent/11.0)

            nx.set_node_attributes(G, cent, 'betweenness')

            clus = nx.clustering(G, weight='weight')
            
            avg_clus = 0
            for i in clus:
                avg_clus += clus[i]
            print('Average Clustering Coefficient = ', avg_clus/11.0)

            nx.set_node_attributes(G, clus, 'Cluster')

            team_json = json_graph.node_link_data(G)

            with open("sample_weight_clus.json", "w") as outfile: 
                json.dump(team_json, outfile)

            display(IFrame(src='./d3_tip.html', width=900, height=600))

        #Mark *********************************************
#         teamId = events.iloc[0]['teamId']

#         lineup = target_match.iloc[0]['teamsData']

#         first_11 = lineup[str(teamId)]['formation']['lineup']
    
#         players_id = []
#         players_dict = {}
#         players_dict_ulta = {}
#         for i in range(len(first_11)):
        
#             players_id.append(first_11[i]['playerId'])
#             players_dict[first_11[i]['playerId']] = i
#             players_dict_ulta[i] = first_11[i]['playerId']

#         players_names = {}
#         for i in range(11):
#             pid = players_dict_ulta[i]
#             players_names[i] = players[players['wyId']==pid].iloc[0]['shortName']
        
#         team_pass = events[events['eventName']=='Pass']
#         team_pass = team_pass[team_pass['playerId'].isin(players_id)]
#         team_pass['Length'] = team_pass.tags.apply(lambda x: len(x))
#         team_pass = team_pass[team_pass['Length']==1]
#         team_pass['accurate'] = team_pass.apply(lambda row: row['tags'][0]['id'], axis=1)
#         team_pass = team_pass[team_pass['accurate']==1801]
#         team_pass = team_pass.reset_index(drop=True)
        
#         players_matrix = np.zeros((11, 11), dtype=int)
        
#         for i in range(1, len(team_pass)):
            
#             if team_pass.loc[i, 'eventSec']-team_pass.loc[i-1, 'eventSec'] < 5.0:
                
#                 player_from = team_pass.loc[i-1, 'playerId']
#                 player_to = team_pass.loc[i, 'playerId']
                
#                 if player_from != player_to:
#                     players_matrix[players_dict[player_from]][players_dict[player_to]] += 1
#                     players_matrix[players_dict[player_to]][players_dict[player_from]] += 1
                
        
#         G = nx.from_numpy_matrix(players_matrix)
        
#         G = nx.relabel_nodes(G, players_names)
        
#         nx.draw(G, with_labels=True)   #Does not show anything

#         cent = nx.algorithms.centrality.betweenness_centrality(G, weight='weight')
        
#         nx.set_node_attributes(G, cent, 'betweenness')
        
#         clus = nx.clustering(G, weight='weight')
        
#         nx.set_node_attributes(G, clus, 'Cluster')
        
#         team_json = json_graph.node_link_data(G)
        
#         with open("sample_weight_clus.json", "w") as outfile: 
#             json.dump(team_json, outfile)
            
#         display(IFrame(src='./d3_tip.html', width=900, height=600))

    #invoke team selector : get the ball rolling
    interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

In [14]:
dashboard_graph_both()

A Jupyter Widget

## Visualizing both the opponents

In [305]:
from ipywidgets import interact, interactive, interact_manual, widgets, Layout
import warnings 
warnings.filterwarnings('ignore')

def dashboard_graph_both_opp():
    #global variables whose values will be updated with every successive selection : filtering achieved through modification of these global vars
    team_1, team_2 = None, None
    matches = None
    target_match = None
    target_team = None
    target_player = None
    events = None
    selected_events = None
    teamData = None
    target_match_not = None

    #First selector that user sees is team selector : get its options
    england_teams = pd.DataFrame(columns=teams.columns)
    for index,row in teams.iterrows():
        if row['area']['name'] == 'England':
            england_teams = england_teams.append(row)
    team_selector_options = sorted(england_teams['name'].tolist())

    #create team selectors - first interactive element to be called, invocation is on last line 
    team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
    team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

    def select_teams(team_1_selector, team_2_selector):
        global team_1, team_2, matches

        team_1 = team(team_1_selector)
        team_2 = team(team_2_selector)
        matches = team_1.get_matches(team_2)

        #teams selected, call match selector to select one of 2 matches between them
        match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match:', layout=Layout(margin='0px 0px 0px -3px'))
        interact(select_match, match_selector=match_selector)

    def select_match(match_selector) -> pd.DataFrame:
        global matches, target_match, team_1, team_2, target_match_not

        target_match = matches[matches['label'] == match_selector]
        target_match_not = matches[matches['label'] != match_selector]
#         display(target_match) #for debugging
    
        #match selected, select which team in match to analyse
        target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team:', layout=Layout(margin='0px 0px 0px -8px'))
        interact(select_team, team_selector=target_team_selector)
        
    def select_team(team_selector):
        global team_1, team_2, target_match, target_team, events, target_match_not

        target_team = team_1 if team_selector == team_1.name else team_2
        target_team_not = team_1 if team_selector != team_1.name else team_2
        
#         print('Before Events')
        print(target_match.iloc[0]['label'])
#         print('Target_not', target_match_not.iloc[0]['label'])

        events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])] 
        events_not = target_team.all_events[target_team.all_events['matchId'] == int(target_match_not['wyId'])]

        #Mark *********************************************
        
        print(target_team.name)
        
        teamId = events.iloc[0]['teamId']

        lineup = target_match.iloc[0]['teamsData']

        first_11 = lineup[str(teamId)]['formation']['lineup']
    
        players_id = []
        players_dict = {}
        players_dict_ulta = {}
        for i in range(len(first_11)):
        
            players_id.append(first_11[i]['playerId'])
            players_dict[first_11[i]['playerId']] = i
            players_dict_ulta[i] = first_11[i]['playerId']

        players_names = {}
        for i in range(11):
            pid = players_dict_ulta[i]
            players_names[i] = players[players['wyId']==pid].iloc[0]['shortName']
        
        team_pass = events[events['eventName']=='Pass']
        team_pass = team_pass[team_pass['playerId'].isin(players_id)]
        team_pass['Length'] = team_pass.tags.apply(lambda x: len(x))
        team_pass = team_pass[team_pass['Length']==1]
        team_pass['accurate'] = team_pass.apply(lambda row: row['tags'][0]['id'], axis=1)
        team_pass = team_pass[team_pass['accurate']==1801]
        team_pass = team_pass.reset_index(drop=True)
        
        players_matrix = np.zeros((11, 11), dtype=int)
        
        for i in range(1, len(team_pass)):
            
            if team_pass.loc[i, 'eventSec']-team_pass.loc[i-1, 'eventSec'] < 5.0:
                
                player_from = team_pass.loc[i-1, 'playerId']
                player_to = team_pass.loc[i, 'playerId']
                
                if player_from != player_to:
                    players_matrix[players_dict[player_from]][players_dict[player_to]] += 1
                    players_matrix[players_dict[player_to]][players_dict[player_from]] += 1
                
        
        G = nx.from_numpy_matrix(players_matrix)
        
        G = nx.relabel_nodes(G, players_names)
        
#         nx.draw(G, with_labels=True)   #Does not show anything

        cent = nx.algorithms.centrality.betweenness_centrality(G, weight='weight')
    
        avg_cent = 0
        for i in cent:
            avg_cent += cent[i]
        print('Average Betwenness = ', avg_cent/11.0)
        
        nx.set_node_attributes(G, cent, 'betweenness')
        
        clus = nx.clustering(G, weight='weight')
        
        avg_clus = 0
        for i in clus:
            avg_clus += clus[i]
        print('Average Clustering Coefficient = ', avg_clus/11.0)
        
        nx.set_node_attributes(G, clus, 'Cluster')
        
        team_json = json_graph.node_link_data(G)
        
        with open("sample_weight_clus.json", "w") as outfile: 
            json.dump(team_json, outfile)
            
#         print(target_team.name)
            
        display(IFrame(src='./d3_tip.html', width=900, height=600))
        
        #2
        
        print(target_team_not.name)
        
        events = target_team_not.all_events[target_team_not.all_events['matchId'] == int(target_match['wyId'])] 
        events_not = target_team.all_events[target_team.all_events['matchId'] == int(target_match_not['wyId'])]

        teamId = events.iloc[0]['teamId']

        lineup = target_match.iloc[0]['teamsData']

        first_11 = lineup[str(teamId)]['formation']['lineup']
    
        players_id = []
        players_dict = {}
        players_dict_ulta = {}
        for i in range(len(first_11)):
        
            players_id.append(first_11[i]['playerId'])
            players_dict[first_11[i]['playerId']] = i
            players_dict_ulta[i] = first_11[i]['playerId']

        players_names = {}
        for i in range(11):
            pid = players_dict_ulta[i]
            players_names[i] = players[players['wyId']==pid].iloc[0]['shortName']
        
        team_pass = events[events['eventName']=='Pass']
        team_pass = team_pass[team_pass['playerId'].isin(players_id)]
        team_pass['Length'] = team_pass.tags.apply(lambda x: len(x))
        team_pass = team_pass[team_pass['Length']==1]
        team_pass['accurate'] = team_pass.apply(lambda row: row['tags'][0]['id'], axis=1)
        team_pass = team_pass[team_pass['accurate']==1801]
        team_pass = team_pass.reset_index(drop=True)
        
        players_matrix = np.zeros((11, 11), dtype=int)
        
        for i in range(1, len(team_pass)):
            
            if team_pass.loc[i, 'eventSec']-team_pass.loc[i-1, 'eventSec'] < 5.0:
                
                player_from = team_pass.loc[i-1, 'playerId']
                player_to = team_pass.loc[i, 'playerId']
                
                if player_from != player_to:
                    players_matrix[players_dict[player_from]][players_dict[player_to]] += 1
                    players_matrix[players_dict[player_to]][players_dict[player_from]] += 1
                
        
        G = nx.from_numpy_matrix(players_matrix)
        
        G = nx.relabel_nodes(G, players_names)
        
#         nx.draw(G, with_labels=True)   #Does not show anything

        cent = nx.algorithms.centrality.betweenness_centrality(G, weight='weight')
    
        avg_cent = 0
        for i in cent:
            avg_cent += cent[i]
        print('Average Betwenness = ', avg_cent/11.0)
        
        nx.set_node_attributes(G, cent, 'betweenness')
        
        clus = nx.clustering(G, weight='weight')
        
        avg_clus = 0
        for i in clus:
            avg_clus += clus[i]
        print('Average Clustering Coefficient = ', avg_clus/11.0)
        
        nx.set_node_attributes(G, clus, 'Cluster')
        
        team_json = json_graph.node_link_data(G)
        
        with open("sample_weight_clus.json", "w") as outfile: 
            json.dump(team_json, outfile)
            
#         print(target_team_not.name)
            
        display(IFrame(src='./d3_tip.html', width=900, height=600))
        
        

    #invoke team selector : get the ball rolling
    interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

In [306]:
dashboard_graph_both_opp()

A Jupyter Widget

## Code below can be ignored

In [177]:
from IPython.display import IFrame

IFrame(src='./d3_copy.html', width=900, height=600)

In [183]:
from IPython.display import IFrame

IFrame(src='./d3_tip.html', width=900, height=600)

In [182]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [188]:
HTML('''<!DOCTYPE html>
<meta charset="utf-8">
<style>

	.link {
	  stroke: #c1c1c1;
    pointer-events: all;
	}

  .node circle {
	  pointer-events: all;
	  stroke: #777;
	  stroke-width: 1px;
	}


	div.tooltip {
    position: absolute;
    background-color: white;
    max-width; 200px;
    height: auto;
    padding: 1px;
    border-style: solid;
    border-radius: 4px;
    border-width: 1px;
    box-shadow: 3px 3px 10px rgba(0, 0, 0, .5);
    pointer-events: none;
  }

</style>
<body>
</body>
<svg width="960" height="700"></svg>
<script src="https://d3js.org/d3.v4.min.js" charset="utf-8"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3-legend/2.24.0/d3-legend.min.js" charset="utf-8"></script>
<script src="https://d3js.org/d3-scale-chromatic.v1.min.js" charset="utf-8"></script>
<script>





var color = d3.scaleOrdinal(d3.schemeSet3);
color(0);
  color(1);
  color(2);
  color(3);
  color(4);
  color(5);
  color(6);
  color(7);
  color(8);
  color(9);
  color(10);



var tooltip = d3.select("body")
	.append("div")
	.attr("class", "tooltip")
	.style("opacity", 0);


d3.json("sample_weight_clus.json", function(error, graph) {
  if (error) throw error;
 	const svg = d3.select('svg'),
        width = +svg.attr('width'),
  			height = +svg.attr('height');



//  const width = 960;
//   const height = 700;

  const simulation = d3.forceSimulation()
    .nodes(graph.nodes)
    .force('link', d3.forceLink().id(d => d.id).distance(linkDistance).strength(0.1))
    .force('charge', d3.forceManyBody().strength(50))
    .force('center', d3.forceCenter(width / 2, height / 2))
    .on('tick', ticked);

  simulation.force('link')
    .links(graph.links);

  const R = 6;




  let link = svg.selectAll('line')
    .data(graph.links)
    .enter().append('line')
      .attr("stroke-width", function(d) { return Math.sqrt(d.weight); });

  link
    .attr('class', 'link')
    // .enter().append('line')
    //   .attr("stroke-width", function(d) { return Math.sqrt(d.weight); })
  	.on('mouseover.tooltip', function(d) {
      	tooltip.transition()
        	.duration(300)
        	.style("opacity", .8);
      	tooltip.html("Source:"+ d.source.id +
                     "<p/>Target:" + d.target.id +
                    "<p/>Strength:"  + d.weight)
        	.style("left", (d3.event.pageX) + "px")
        	.style("top", (d3.event.pageY + 10) + "px");
    	})
    	.on("mouseout.tooltip", function() {
	      tooltip.transition()
	        .duration(100)
	        .style("opacity", 0);
	    })
  		.on('mouseout.fade', fade(1))
	    .on("mousemove", function() {
	      tooltip.style("left", (d3.event.pageX) + "px")
	        .style("top", (d3.event.pageY + 10) + "px");
	    });
;

  let node = svg.selectAll('.node')
    .data(graph.nodes)
    .enter().append('g')
    .attr('class', 'node')
    .call(d3.drag()
    	.on("start", dragstarted)
      .on("drag", dragged)
      .on("end", dragended));;

  node.append('circle')
    .attr('r', R)
  	.attr("fill", function(d) { return color(d.group);})
    .on('mouseover.tooltip', function(d) {
      	tooltip.transition()
        	.duration(300)
        	.style("opacity", .8);
      	tooltip.html("Name:" + d.id + "<p/>Betweenness:" + d.betweenness+ "<p/>Clustering Coefficient:" + d.Cluster)
        	.style("left", (d3.event.pageX) + "px")
        	.style("top", (d3.event.pageY + 10) + "px");
    	})
  	.on('mouseover.fade', fade(0.1))
    .on("mouseout.tooltip", function() {
        tooltip.transition()
	        .duration(100)
	        .style("opacity", 0);
	    })
  	.on('mouseout.fade', fade(1))
	    .on("mousemove", function() {
	      tooltip.style("left", (d3.event.pageX) + "px")
	        .style("top", (d3.event.pageY + 10) + "px");
	    })
  	.on('dblclick',releasenode)


  node.append('text')
    .attr('x', 0)
    .attr('dy', '.35em')
    .text(d => d.id);

  function ticked() {
    link
      .attr('x1', d => d.source.x)
      .attr('y1', d => d.source.y)
      .attr('x2', d => d.target.x)
      .attr('y2', d => d.target.y);

    node
      .attr('transform', d => `translate(${d.x},${d.y})`);
  }

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  // d.fx = null;
  // d.fy = null;
}
function releasenode(d) {
    d.fx = null;
    d.fy = null;
}

function linkDistance(d) {
    // return d.value*30;
  return 500/d.weight;
}

  const linkedByIndex = {};
  graph.links.forEach(d => {
    linkedByIndex[`${d.source.index},${d.target.index}`] = 1;
  });

  function isConnected(a, b) {
    return linkedByIndex[`${a.index},${b.index}`] || linkedByIndex[`${b.index},${a.index}`] || a.index === b.index;
  }

  function fade(opacity) {
    return d => {
      node.style('stroke-opacity', function (o) {
        const thisOpacity = isConnected(d, o) ? 1 : opacity;
        this.setAttribute('fill-opacity', thisOpacity);
        return thisOpacity;
      });

      link.style('stroke-opacity', o => (o.source === d || o.target === d ? 1 : opacity));

    };
  }
  var sequentialScale = d3.scaleOrdinal(d3.schemeSet3)
  .domain([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]);


svg.append("g")
  .attr("class", "legendSequential")
  .attr("transform", "translate("+(width-140)+","+(height-300)+")");

// var legendSequential = d3.legendColor()
//     .shapeWidth(30)
//     .cells(11)
//     .orient("vertical")
// 		.title("Group number by color:")
// 		.titleWidth(100)
//     .scale(sequentialScale)

svg.select(".legendSequential")
  .call(legendSequential);


})



</script>
''')

In [189]:
HTML('<script src="https://d3js.org/d3.v4.min.js"></script>')

In [190]:
css_text = '''.link {
	  stroke: #c1c1c1;
    pointer-events: all;
	}

  .node circle {
	  pointer-events: all;
	  stroke: #777;
	  stroke-width: 1px;
	}


	div.tooltip {
    position: absolute;
    background-color: white;
    max-width; 200px;
    height: auto;
    padding: 1px;
    border-style: solid;
    border-radius: 4px;
    border-width: 1px;
    box-shadow: 3px 3px 10px rgba(0, 0, 0, .5);
    pointer-events: none;
  }'''

In [191]:
js_text_template = Template('''var color = d3.scaleOrdinal(d3.schemeSet3);
color(0);
  color(1);
  color(2);
  color(3);
  color(4);
  color(5);
  color(6);
  color(7);
  color(8);
  color(9);
  color(10);



var tooltip = d3.select("body")
	.append("div")
	.attr("class", "tooltip")
	.style("opacity", 0);


//d3.json("sample_weight_clus.json", function(error, graph) {
  //if (error) throw error;
  
var graph = $python_data;

 	const svg = d3.select('svg'),
        width = +svg.attr('width'),
  			height = +svg.attr('height');



//  const width = 960;
//   const height = 700;

  const simulation = d3.forceSimulation()
    .nodes(graph.nodes)
    .force('link', d3.forceLink().id(d => d.id).distance(linkDistance).strength(0.1))
    .force('charge', d3.forceManyBody().strength(50))
    .force('center', d3.forceCenter(width / 2, height / 2))
    .on('tick', ticked);

  simulation.force('link')
    .links(graph.links);

  const R = 6;




  let link = svg.selectAll('line')
    .data(graph.links)
    .enter().append('line')
      .attr("stroke-width", function(d) { return Math.sqrt(d.weight); });

  link
    .attr('class', 'link')
    // .enter().append('line')
    //   .attr("stroke-width", function(d) { return Math.sqrt(d.weight); })
  	.on('mouseover.tooltip', function(d) {
      	tooltip.transition()
        	.duration(300)
        	.style("opacity", .8);
      	tooltip.html("Source:"+ d.source.id +
                     "<p/>Target:" + d.target.id +
                    "<p/>Strength:"  + d.weight)
        	.style("left", (d3.event.pageX) + "px")
        	.style("top", (d3.event.pageY + 10) + "px");
    	})
    	.on("mouseout.tooltip", function() {
	      tooltip.transition()
	        .duration(100)
	        .style("opacity", 0);
	    })
  		.on('mouseout.fade', fade(1))
	    .on("mousemove", function() {
	      tooltip.style("left", (d3.event.pageX) + "px")
	        .style("top", (d3.event.pageY + 10) + "px");
	    });
;

  let node = svg.selectAll('.node')
    .data(graph.nodes)
    .enter().append('g')
    .attr('class', 'node')
    .call(d3.drag()
    	.on("start", dragstarted)
      .on("drag", dragged)
      .on("end", dragended));;

  node.append('circle')
    .attr('r', R)
  	.attr("fill", function(d) { return color(d.group);})
    .on('mouseover.tooltip', function(d) {
      	tooltip.transition()
        	.duration(300)
        	.style("opacity", .8);
      	tooltip.html("Name:" + d.id + "<p/>Betweenness:" + d.betweenness+ "<p/>Clustering Coefficient:" + d.Cluster)
        	.style("left", (d3.event.pageX) + "px")
        	.style("top", (d3.event.pageY + 10) + "px");
    	})
  	.on('mouseover.fade', fade(0.1))
    .on("mouseout.tooltip", function() {
        tooltip.transition()
	        .duration(100)
	        .style("opacity", 0);
	    })
  	.on('mouseout.fade', fade(1))
	    .on("mousemove", function() {
	      tooltip.style("left", (d3.event.pageX) + "px")
	        .style("top", (d3.event.pageY + 10) + "px");
	    })
  	.on('dblclick',releasenode)


  node.append('text')
    .attr('x', 0)
    .attr('dy', '.35em')
    .text(d => d.id);

  function ticked() {
    link
      .attr('x1', d => d.source.x)
      .attr('y1', d => d.source.y)
      .attr('x2', d => d.target.x)
      .attr('y2', d => d.target.y);

    node
      .attr('transform', d => `translate(${d.x},${d.y})`);
  }

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  // d.fx = null;
  // d.fy = null;
}
function releasenode(d) {
    d.fx = null;
    d.fy = null;
}

function linkDistance(d) {
    // return d.value*30;
  return 500/d.weight;
}

  const linkedByIndex = {};
  graph.links.forEach(d => {
    linkedByIndex[`${d.source.index},${d.target.index}`] = 1;
  });

  function isConnected(a, b) {
    return linkedByIndex[`${a.index},${b.index}`] || linkedByIndex[`${b.index},${a.index}`] || a.index === b.index;
  }

  function fade(opacity) {
    return d => {
      node.style('stroke-opacity', function (o) {
        const thisOpacity = isConnected(d, o) ? 1 : opacity;
        this.setAttribute('fill-opacity', thisOpacity);
        return thisOpacity;
      });

      link.style('stroke-opacity', o => (o.source === d || o.target === d ? 1 : opacity));

    };
  }
  var sequentialScale = d3.scaleOrdinal(d3.schemeSet3)
  .domain([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]);


svg.append("g")
  .attr("class", "legendSequential")
  .attr("transform", "translate("+(width-140)+","+(height-300)+")");

// var legendSequential = d3.legendColor()
//     .shapeWidth(30)
//     .cells(11)
//     .orient("vertical")
// 		.title("Group number by color:")
// 		.titleWidth(100)
//     .scale(sequentialScale)

svg.select(".legendSequential")
  .call(legendSequential);


})''')

In [192]:
html_template = Template('''
<style>$css_text</style>
<div id="graph-div"></div>
<script>$js_text</script>
''')

In [198]:
f = open('sample_weight_clus.json',) 
req_json = json.loads(f.read())

In [199]:
req_json

{'directed': False,
 'graph': {},
 'links': [{'source': 'R. Holding', 'target': 'Mohamed Elneny', 'weight': 16},
  {'source': 'R. Holding', 'target': 'D. Welbeck', 'weight': 1},
  {'source': 'R. Holding', 'target': 'S. Kola\\u0161inac', 'weight': 3},
  {'source': 'R. Holding', 'target': 'A. Lacazette', 'weight': 1},
  {'source': 'R. Holding', 'target': 'A. Oxlade-Chamberlain', 'weight': 1},
  {'source': 'R. Holding', 'target': 'Nacho Monreal', 'weight': 17},
  {'source': 'R. Holding', 'target': 'Beller\\u00edn', 'weight': 15},
  {'source': 'R. Holding', 'target': 'M. \\u00d6zil', 'weight': 8},
  {'source': 'R. Holding', 'target': 'P. \\u010cech', 'weight': 3},
  {'source': 'R. Holding', 'target': 'G. Xhaka', 'weight': 8},
  {'source': 'Mohamed Elneny', 'target': 'D. Welbeck', 'weight': 3},
  {'source': 'Mohamed Elneny', 'target': 'S. Kola\\u0161inac', 'weight': 9},
  {'source': 'Mohamed Elneny', 'target': 'A. Lacazette', 'weight': 2},
  {'source': 'Mohamed Elneny', 'target': 'A. Oxlade

In [203]:
js_text = js_text_template.substitute({'python_data': req_json})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

ValueError: Invalid placeholder in string: line 129, col 42

In [1]:
from IPython.display import IFrame

In [9]:
IFrame(src='./d3_pitch.html', width=1200, height=600)